# Chapter 9 — Tracing

The Agents SDK ships with **built-in tracing** that records everything in a run: LLM generations, tool calls, handoffs, guardrails, plus your own metadata. In the Traces UI you can debug and monitor workflows across dev and prod.

---

## 1. Enable/disable tracing

Tracing is **ON by default**. You can turn it off in three ways:

**A. Environment-wide**
```bash
# disable everywhere
export OPENAI_AGENTS_DISABLE_TRACING=1
```


**B. Per file / process**

```python

from agents import set_tracing_disabled
set_tracing_disabled(True)   # disable for this process
# set_tracing_disabled(False)  # re-enable

```


**B. Per file / process**

```python
from agents import Runner, RunConfig
result = await Runner.run(
    agent,
    input="...",
    run_config=RunConfig(tracing_disabled=True),  # only this run is untraced
)

```

> If tracing is enabled and you set OPENAI_API_KEY, traces are exported.
Without a key you may see: OPENAI_API_KEY is not set, skipping trace export.


## 2. Traces & Spans (concepts)

**Traces** are end-to-end records of a whole “workflow run,” made up of multiple spans. A trace carries:

- **workflow_name** — The logical workflow or app name (e.g., “code_generation”, “customer_support”).  
- **trace_id** — The unique ID for this trace. If you don’t provide one, we’ll mint it for you.  
  Must match the format: `trace_<32_alphanumeric>` (because chaos needs a little order).  
- **group_id** — Optional grouping key to link multiple traces from the *same session* (e.g., a chat thread ID).  
- **disabled** — If `True`, the trace won’t be saved (useful when you want silence, not surveillance).  
- **metadata** — Optional key–value tags for filtering, dashboards, and future-you.

**Spans** are time-boxed units of work inside a trace. Each span includes:

- **started_at** / **ended_at** — Timestamps to mark its glorious beginning and dignified end.  
- **trace_id** — Which trace this span belongs to (family matters).  
- **parent_id** — Optional pointer to the parent span (so you can build the family tree).  
- **span_data** — Details about the operation. Examples:  
  - `AgentSpanData` — Info about the agent that ran.  
  - `GenerationSpanData` — LLM generation specifics.  
  - (…and other span flavors you’ll meet along the way.)

*TL;DR:* Traces are the movie; spans are the scenes. Roll camera. 🎬

## 3. Default Tracing (what the SDK records for you)

Out of the box, the SDK traces the usual suspects—so you don’t have to:

- **Whole run**  
  Every `Runner.{run, run_sync, run_streamed}()` is wrapped in a **`trace()`**. One trace to rule them all.

- **Agent execution**  
  Each agent turn gets an **`agent_span()`**. (Lights, camera, agent.)

- **LLM generations**  
  Model outputs live inside **`generation_span()`**. Token drama included.

- **Function tools**  
  Each tool call is wrapped in a **`function_span()`**. Handy for “who-called-what-and-why”.

- **Guardrails**  
  When a safety check trips, you’ll see a **`guardrail_span()`**. Seatbelts on.

- **Handoffs**  
  Delegations show up as **`handoff_span()`**. Pass the baton, keep the trace.

- **Speech I/O**  
  - Speech → text: **`transcription_span()`**  
  - Text → speech: **`speech_span()`**  
  - Related audio spans may be grouped under **`speech_group_span()`** (the band plays together).

### Naming & customization
- By default, traces are named **"Agent trace"**.  
- Use **`trace(...)`** or per-run **`RunConfig`** to set a custom **`workflow_name`**, **`trace_id`**, **`group_id`**, and **`metadata`** (because dashboards love tidy labels).

### Bring-your-own sink (optional)
You can register a **custom trace processor** to export traces elsewhere—either as an alternative to the default store or as an additional sink. Think: ship to your log pipeline, data lake, or observability stack. (Your future debugging self will thank you.)


## 4. Advanced tracing — grouping multiple runs under one trace

Sometimes you want **several `Runner.run(...)` calls** to appear under the **same trace** (one “story” with multiple “scenes”).  
Wrap those calls in the `trace(...)` context manager. Everything inside becomes spans of a **single trace**.

In [7]:
from agents import Agent, Runner, trace, RunConfig, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
from dotenv import load_dotenv

# --- Env & model ---
load_dotenv()
api_key = os.getenv("API_KEY")
llm = LitellmModel(
    model="gpt-4.1-nano-2025-04-14",
    api_key=api_key,
    base_url="https://api.openai.com/v1",
)
set_tracing_disabled(True)
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY", api_key)  # set if missing
agent = Agent(name="Joke generator", instructions="Tell funny jokes.")

# In a notebook cell:
with trace("Joke workflow"):                      # one trace for both calls
    # If your RunConfig supports these, they’re safe to pass:
    first = await Runner.run(
        agent,
        "Tell me a joke",
        run_config=RunConfig(workflow_name="Joke workflow", group_id="demo_thread_1")
    )
    print("[STEP=generate] done")

    second = await Runner.run(
        agent,
        f"Rate this joke: {first.final_output}",
        run_config=RunConfig(workflow_name="Joke workflow", group_id="demo_thread_1")
    )
    print("[STEP=rate] done")

print("Joke:", first.final_output)
print("Rating:", second.final_output)

[STEP=generate] done
[STEP=rate] done
Joke: Why did the scarecrow win an award?

Because he was outstanding in his field! 🌾😄
Rating: That’s a **classic**! I'd rate it a **solid 8/10** for timeless pun-appeal and lightheartedness. 🌾😂

Pros:
- Universal and family-friendly
- Smooth wordplay with “outstanding in his field”
- Delightful visual image

Cons:
- You might have heard it before!

Want another corny (get it?) joke?
